In [1]:
import requests
import pandas as pd
import time
import json

from requests.auth import HTTPBasicAuth

In [2]:
USERNAME = "feenionloyed"
PASSWORD = "2001adp@549"

In [3]:
# Store all data
all_boards = []
all_sprints = []
all_issues = []
all_backlog_issues = []

In [4]:
# Get all boards
def fetch_all_boards():
    boards = []
    start_at = 0
    max_results = 50
    is_last = False

    while not is_last:
        url = f"https://issues.apache.org/jira/rest/agile/1.0/board?startAt={start_at}&maxResults={max_results}"
        response = requests.get(url, auth=HTTPBasicAuth(USERNAME, PASSWORD))

        if response.status_code != 200:
            print(f"Error fetching boards: {response.status_code}")
            break

        data = response.json()
        boards.extend(data['values'])
        is_last = data['isLast']
        start_at += max_results
        time.sleep(1)  # Respect rate limits

    return boards


In [5]:
# Get sprints for a board
def fetch_sprints(board_id):
    sprints = []
    start_at = 0
    max_results = 50
    is_last = False

    while not is_last:
        url = f"https://issues.apache.org/jira/rest/agile/1.0/board/{board_id}/sprint?startAt={start_at}&maxResults={max_results}"
        response = requests.get(url, auth=HTTPBasicAuth(USERNAME, PASSWORD))

        if response.status_code != 200:
            # Some boards may not have sprints or might be inaccessible
            print(f"Error fetching sprints for board {board_id}: {response.status_code}")
            break

        try:
            data = response.json()
            sprints.extend(data['values'])
            is_last = data.get('isLast', True)
            start_at += max_results
        except:
            break

        time.sleep(1)  # Respect rate limits

    return sprints


In [6]:
# Get issues for a sprint
def fetch_sprint_issues(board_id, sprint_id):
    issues = []
    start_at = 0
    max_results = 50
    is_last = False

    while not is_last:
        url = f"https://issues.apache.org/jira/rest/agile/1.0/board/{board_id}/sprint/{sprint_id}/issue?startAt={start_at}&maxResults={max_results}"
        response = requests.get(url, auth=HTTPBasicAuth(USERNAME, PASSWORD))

        if response.status_code != 200:
            print(f"Error fetching issues for board {board_id}, sprint {sprint_id}: {response.status_code}")
            break

        try:
            data = response.json()
            issues.extend(data['issues'])
            is_last = data.get('isLast', True)
            start_at += max_results
        except:
            break

        time.sleep(1)  # Respect rate limits

    return issues



In [7]:
# Get backlog issues
def fetch_backlog_issues(board_id):
    issues = []
    start_at = 0
    max_results = 50
    is_last = False

    while not is_last:
        url = f"https://issues.apache.org/jira/rest/agile/1.0/board/{board_id}/backlog?startAt={start_at}&maxResults={max_results}"
        response = requests.get(url, auth=HTTPBasicAuth(USERNAME, PASSWORD))

        if response.status_code != 200:
            print(f"Error fetching backlog for board {board_id}: {response.status_code}")
            break

        try:
            data = response.json()
            issues.extend(data['issues'])
            is_last = data.get('isLast', True)
            start_at += max_results
        except:
            break

        time.sleep(1)  # Respect rate limits

    return issues



In [8]:
print("Fetching all boards...")
boards = fetch_all_boards();
print(f"Found {len(boards)} boards")
# print(boards)

Fetching all boards...
Found 251 boards


In [9]:
boards = boards[:10]
# print(boards)

In [15]:
for board in boards:
    board_id = board['id']

    if(board["type"] == "scrum"):
        print(f"Processing board: {board['name']} (ID: {board_id})")
        # Get sprints
        sprints = fetch_sprints(board_id)
        print(f"Found {len(sprints)} sprints")

        for sprint in sprints:
            sprint['boardId'] = board_id
            all_sprints.append(sprint)

            # Get issues for this sprint
            sprint_id = sprint['id']
            issues = fetch_sprint_issues(board_id, sprint_id)
            print(f"Found {len(issues)} issues")

            for issue in issues:
                try:
                    dummy_issue = {}
                    dummy_issue['boardId'] = board_id
                    dummy_issue['sprintId'] = sprint_id
                    dummy_issue['key'] = issue['key']
                    dummy_issue['priority'] = issue['priority']['name']
                    dummy_issue['status'] = issue['status']['name']
                    dummy_issue['status_category'] = issue['status']['statusCategory']['name']
                    dummy_issue['creator'] = issue['creator']['displayName']
                    dummy_issue['reporter'] = issue['reporter']['displayName']
                    dummy_issue['closedSprints'] = issue['closedSprints']['id']
                    dummy_issue['progress'] = issue['progress']['progress']
                    dummy_issue['progress_total'] = issue['progress']['total']
                    dummy_issue['worklog'] = issue['progress']['total']
                    dummy_issue['issuetype'] = issue['issuetype']['name']
                    dummy_issue['project'] = issue['project']['name']
                    dummy_issue['created'] = issue['created']
                    dummy_issue['updated'] = issue['updated']
                    dummy_issue['description'] = issue['description']
                    dummy_issue['summary'] = issue['summary']
                    dummy_issue['duedate'] = issue['duedate']
                    print(dummy_issue)
                    all_issues.append(dummy_issue)
                except:
                    continue




        # Get backlog issues
        backlog = fetch_backlog_issues(board_id)
        print(f"Found {len(backlog)} backlog issues")
        for issue in backlog:
            dummy_backlog_issue = {}
            dummy_backlog_issue['boardId'] = board_id
            dummy_backlog_issue['isBacklog'] = True


            fix_versions = issue["fields"].get("fixVersions", [])
            dummy_backlog_issue['fixVersion'] = fix_versions[0]["name"] if fix_versions else None
            dummy_backlog_issue['priority'] = issue["fields"].get("priority", {}).get("name")
            dummy_backlog_issue['assignee'] = issue["fields"].get("assignee", {}).get("displayName")
            dummy_backlog_issue['status'] = issue["fields"].get("status", {}).get("name")
            dummy_backlog_issue['status_category'] = issue["fields"].get("status", {}).get("status_category", {}).get("name")
            dummy_backlog_issue['creator'] = issue["fields"].get("creator", {}).get("displayName")
            dummy_backlog_issue['reporter'] = issue["fields"].get("reporter", {}).get("displayName")
            dummy_backlog_issue['issuetype'] = issue["fields"].get("issuetype", {}).get("name")
            dummy_backlog_issue['project'] = issue["fields"].get("project", {}).get("name")
            dummy_backlog_issue['created'] = issue["fields"].get("created")
            dummy_backlog_issue['updated'] = issue["fields"].get("updated")
            dummy_backlog_issue['description'] = issue["fields"].get("description")
            dummy_backlog_issue['summary'] = issue["fields"].get("summary")


            all_backlog_issues.append(dummy_backlog_issue)
            print(dummy_backlog_issue)
            print("-------------------")

Processing board: 0.6.1 (ID: 365)
Found 0 sprints
Found 50 backlog issues
[]
-------------------
[]
-------------------
[]
-------------------
[]
-------------------
[]
-------------------
[]
-------------------
[]
-------------------
[]
-------------------
[]
-------------------
[]
-------------------
[]
-------------------
[{'self': 'https://issues.apache.org/jira/rest/api/2/version/12347241', 'id': '12347241', 'description': 'Next Release of the rel/0.6 branch', 'name': '0.6.1', 'archived': False, 'released': True, 'releaseDate': '2020-03-30'}]
-------------------
[{'self': 'https://issues.apache.org/jira/rest/api/2/version/12347241', 'id': '12347241', 'description': 'Next Release of the rel/0.6 branch', 'name': '0.6.1', 'archived': False, 'released': True, 'releaseDate': '2020-03-30'}]
-------------------
[{'self': 'https://issues.apache.org/jira/rest/api/2/version/12347241', 'id': '12347241', 'description': 'Next Release of the rel/0.6 branch', 'name': '0.6.1', 'archived': False, 

In [33]:
# Save to files
with open('apache_boards.json', 'w') as f:
    json.dump(all_boards, f)

with open('apache_sprints.json', 'w') as f:
    json.dump(all_sprints, f)

with open('apache_sprint_issues.json', 'w') as f:
    json.dump(all_issues, f)

with open('apache_backlog_issues.json', 'w') as f:
    json.dump(all_backlog_issues, f)

In [34]:
# Create DataFrames for analysis
boards_df = pd.json_normalize(all_boards)
sprints_df = pd.json_normalize(all_sprints)
issues_df = pd.json_normalize(all_issues)
backlog_df = pd.json_normalize(all_backlog_issues)

In [31]:
# boards_df = pd.DataFrame(all_boards);
# sprints_df = pd.DataFrame(all_sprints);
# issues_df = pd.DataFrame(all_issues);
# backlog_df = pd.DataFrame(all_backlog_issues);

In [35]:
# Save as CSV
boards_df.to_csv('./data/apache_boards.csv', index=False)
sprints_df.to_csv('./data/apache_sprints.csv', index=False)
issues_df.to_csv('./data/apache_sprint_issues.csv', index=False)
backlog_df.to_csv('./data/apache_backlog_issues.csv', index=False)